## Deliverable 1: Create a Weather Database

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import os
import sys
import requests

In [3]:
parent_folder = os.path.dirname(os.getcwd())
sys.path.append(parent_folder)

In [4]:
from config import weather_api_key
from citipy import citipy
from datetime import datetime

In [5]:
def get_cities_list(size=2000):
    """
    Get random list of cities 
    
    args:
        size: number of random cities to return
        
    returns:
        cities: list of nearest cities generated from tuple of random latitude and longitude values
    """
    lats = np.random.uniform(low=-90, high=90.0, size=size)
    lngs = np.random.uniform(low=-180, high=180, size=size)
    
    coordinates = list(zip(lats, lngs))
    cities = []
    
    # Identify the nearest city for each latitude and longitude combination
    for coordinate in coordinates:
        city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
        # add to cities list
        if city not in cities:
            cities.append(city)
            
    return cities 

## Generate Random World Cities

In [6]:
#cities = get_cities_list(2000)
cities = ["Boston", "New York", "Bethesda", "Chicago", "Tampa"]

## Get City Weather Data

In [10]:
def read_city_weather_data(base_url):
    """
    Iterage through each city in cities list passed in. For each city, call OpenWeather API for weather data
    
    args:
        base_url: base url for OpenWeather API call
        cities: list of cities to retrieve weather conditions on
    
    returns:
        dictionary of weather data, yielding one city at a time
    """
    
    for i, city in enumerate(cities):
            try:
                city_arg = city.replace(" ","+")
                city_url = f"{base_url}&q={city_arg}"
                city_weather = requests.get(city_url).json()

                yield {
                        "City": city.title(),
                        "Lat": city_weather["coord"]["lat"],
                        "Lng": city_weather["coord"]["lon"],
                        "Max Temp": city_weather["main"]["temp_max"],
                        "Humidity": city_weather["main"]["humidity"],
                        "Cloudiness": city_weather["clouds"]["all"],
                        "Wind Speed": city_weather["wind"]["speed"],
                        "Country": city_weather["sys"]["country"],
                        "Date": datetime.utcfromtimestamp(city_weather["dt"]).strftime("%Y-%m-%d %H:%M:S"),
                        "Current Description": city_weather["weather"][0]["description"]
                    }             
            except:
                print(f"Weather data not available for {city}")
                continue


In [11]:
base_url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [12]:
city_weather_list = []
for w in read_city_weather_data(base_url):
    city_weather_list.append(w)

In [13]:
city_weather_df = pd.DataFrame(city_weather_list)

In [14]:
new_col_order = ['City', 'Country', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness', 'Wind Speed', 'Current Description']
city_weather_df = city_weather_df[new_col_order]

In [15]:
city_weather_df.set_index("City", inplace=True)

In [16]:
city_weather_df.head(25)

,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
City,,,,,,,,
Boston,US,42.3584,-71.0598,73.58,59,57,7.00,broken clouds
New York,US,40.7143,-74.0060,79.95,47,1,11.50,clear sky
Bethesda,US,38.9807,-77.1003,80.22,47,1,8.01,clear sky
Chicago,US,41.8500,-87.6500,82.99,47,4,4.00,clear sky
Tampa,US,27.9475,-82.4584,88.95,56,75,1.01,broken clouds


In [17]:
city_weather_df.to_csv('WeatherPy_Database.csv')